In [1]:
# import all libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import re

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import scale
from sklearn.feature_selection import RFE
from sklearn import model_selection
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import classification_report

from sklearn.preprocessing import Normalizer,StandardScaler
from sklearn.preprocessing.data import normalize

from statistics import mean 

df = pd.read_csv("sales_data_2015.csv")
df.info()

C:\Users\folke\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.preprocessing.data module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.preprocessing. Anything that cannot be imported from sklearn.preprocessing is now part of the private API.
  warnings.warn(message, FutureWarning)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84768 entries, 0 to 84767
Data columns (total 27 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  84768 non-null  int64  
 1   Sale_id     84768 non-null  int64  
 2   bbl_id      84768 non-null  int64  
 3   year        84768 non-null  int64  
 4   borough     84768 non-null  int64  
 5   bldg_ctgy   84768 non-null  object 
 6   tax_cls_p   84765 non-null  object 
 7   block       84768 non-null  int64  
 8   lot         84768 non-null  int64  
 9   easmnt      1 non-null      object 
 10  bldg_cls_p  84765 non-null  object 
 11  address     84768 non-null  object 
 12  apt         20983 non-null  object 
 13  zip         84768 non-null  int64  
 14  res_unit    84768 non-null  int64  
 15  com_unit    84768 non-null  int64  
 16  tot_unit    84768 non-null  int64  
 17  land_sqft   84768 non-null  int64  
 18  tot_sqft    84768 non-null  int64  
 19  yr_built    84768 non-nul

C:\Users\folke\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
df.head()

,Unnamed: 0,Sale_id,bbl_id,year,borough,bldg_ctgy,tax_cls_p,block,lot,easmnt,...,land_sqft,tot_sqft,yr_built,tax_cls_s,bldg_cls_s,sale_date,price,usable,long,lat
0,0,1,2290528,2015,2,01 ONE FAMILY DWELLINGS,1,2905,28,NaN,...,4750,2619,1899,1,A9,08-07-15,0,N,-73.901338,40.841229
1,1,2,2290528,2015,2,01 ONE FAMILY DWELLINGS,1,2905,28,NaN,...,4750,2619,1899,1,A9,08-07-15,0,N,-73.901338,40.841229
2,2,3,230352,2015,2,01 ONE FAMILY DWELLINGS,1,3035,2,NaN,...,1287,2528,1899,1,S1,12/18/2015,0,N,-73.898331,40.848648
3,3,4,2303936,2015,2,01 ONE FAMILY DWELLINGS,1,3039,36,NaN,...,2500,2452,1910,1,S1,08/18/2015,0,N,-73.893338,40.856136
4,4,5,2303936,2015,2,01 ONE FAMILY DWELLINGS,1,3039,36,NaN,...,2500,2452,1910,1,S1,07-01-15,400000,Y,-73.893338,40.856136


In [3]:
df.columns

Index(['Unnamed: 0', 'Sale_id', 'bbl_id', 'year', 'borough', 'bldg_ctgy',
       'tax_cls_p', 'block', 'lot', 'easmnt', 'bldg_cls_p', 'address', 'apt',
       'zip', 'res_unit', 'com_unit', 'tot_unit', 'land_sqft', 'tot_sqft',
       'yr_built', 'tax_cls_s', 'bldg_cls_s', 'sale_date', 'price', 'usable',
       'long', 'lat'],
      dtype='object')

In [4]:
# cleaning option 1
# df2 = df[['borough', 'bldg_cls_p', 'tax_cls_p', 'tot_sqft', 'land_sqft', 'tot_unit', 'res_unit', 'com_unit', 'price']]
# df2 = df2[(df2.tot_sqft != 0) & (df2.tot_sqft != 1) & (df2.price >= 100) & (df2.land_sqft != 0)]

# df2.dropna(subset = ["bldg_cls_p"], inplace=True)
# df2.dropna(subset = ["tax_cls_p"], inplace=True)

# print("Length of dataset is {}".format(len(df2)))

# def bldg_cls_simple(building):
#     return building[0]

# def tax_cls_simple(building):
#     return building[0]

# df2['cat'] = df2.bldg_cls_p.apply(lambda building: bldg_cls_simple(str(building)))
# df2['tax_cat'] = df2.tax_cls_p.apply(lambda building: tax_cls_simple(str(building)))

# # df2 = pd.concat([df2, pd.get_dummies(df2.tax_cls_p)], axis=1)
# df2 = pd.concat([df2, pd.get_dummies(df2.cat)], axis=1)
# df2 = pd.concat([df2, pd.get_dummies(df2.tax_cat)], axis=1)
# df2 = pd.concat([df2, pd.get_dummies(df2.borough, prefix="b")], axis=1)

# df2 = df2.drop(columns=['bldg_cls_p', 'borough', 'tax_cls_p', 'cat', 'tax_cat'])

In [5]:
# cleaning option 2 (all possible features)
df2 = df[['borough', 'bldg_cls_p', 'tax_cls_p','bldg_cls_s', 'tax_cls_s', 'tot_sqft', 'land_sqft', 'tot_unit', 'res_unit', 'com_unit', 'price']]
df2 = df2[(df2.tot_sqft != 0) & (df2.tot_sqft != 1) & (df2.price >= 100) & (df2.land_sqft != 0)]

df2.dropna(subset = ["bldg_cls_p"], inplace=True)
df2.dropna(subset = ["tax_cls_p"], inplace=True)

print("Length of dataset is {}".format(len(df2)))

df2 = pd.concat([df2, pd.get_dummies(df2.tax_cls_p, prefix="tax_cls_p")], axis=1)
df2 = pd.concat([df2, pd.get_dummies(df2.bldg_cls_p, prefix="bdgp_cls_p")], axis=1)
df2 = pd.concat([df2, pd.get_dummies(df2.tax_cls_s, prefix="tax_cls_s")], axis=1)
df2 = pd.concat([df2, pd.get_dummies(df2.bldg_cls_s, prefix="bdgp_cls_s")], axis=1)
df2 = pd.concat([df2, pd.get_dummies(df2.borough, prefix="b")], axis=1)

df2 = df2.drop(columns=['borough', 'bldg_cls_p', 'tax_cls_p','bldg_cls_s'])

Length of dataset is 28070


In [6]:
len(df2.columns)

290

In [7]:
X = df2.drop(columns=['price'])
Y = df2.price
Y = np.array(Y).reshape(-1)
print(X.shape,Y.shape)

(28070, 289) (28070,)


In [8]:
# for column in X.columns:
#     corry = np.corrcoef(Y, X["{}".format(column)])[0][1]
#     if corry < 0.1 and corry > -0.1:
#         print("removed column {0}, because {1}".format(column, corry))
#         X = X.drop(columns=[column])

In [11]:
from sklearn import linear_model

# X = Normalizer().fit_transform(X)
# X = StandardScaler().fit_transform(X)

parameters = {'alpha': list(np.linspace(0.01, 1, 20))}

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = .20, random_state=40)
print(X_train.shape, X_test.shape)

lm = linear_model.Lasso(tol=1.0, normalize=True)

clf = GridSearchCV(lm, parameters)
clf.fit(X_train, y_train)

(22456, 289) (5614, 289)


GridSearchCV(estimator=Lasso(normalize=True, tol=1.0),
             param_grid={'alpha': [0.01, 0.06210526315789474,
                                   0.11421052631578947, 0.16631578947368422,
                                   0.21842105263157896, 0.2705263157894737,
                                   0.32263157894736844, 0.37473684210526315,
                                   0.4268421052631579, 0.4789473684210527,
                                   0.5310526315789474, 0.5831578947368421,
                                   0.6352631578947369, 0.6873684210526316,
                                   0.7394736842105263, 0.791578947368421,
                                   0.8436842105263158, 0.8957894736842106,
                                   0.9478947368421053, 1.0]})

In [12]:
# lm = LinearRegression(normalize=True)
# lm.fit(X_train, y_train)

lm = linear_model.Lasso(tol=10, normalize=True, alpha=0.01)
lm.fit(X_train, y_train)

print('Train:', lm.score(X_train, y_train))
print('Test:', lm.score(X_test, y_test))

Train: 0.6053512448672322
Test: 0.5860313696434315
